In [ ]:
from define import chandefs as chdef
from define import chanreps as chrep
import numpy as np

In [ ]:
# Random channel in its Kraus representation
kraus = chdef.GetKraussForChannel("rand", 0.7)
print(kraus)

In [ ]:
# Converting the Kraus to Pauli transfer matrix
ptm_from_kraus = chrep.ConvertRepresentations(kraus, "krauss", "process")
print(ptm_from_kraus)

In [ ]:
# Converting to Choi
choi = chrep.ConvertRepresentations(kraus, "krauss", "choi")
# and then from Choi to Pauli transfer matrix
ptm_from_choi = chrep.ConvertRepresentations(choi, "choi", "process")
print(ptm_from_choi)

In [ ]:
# Norm of the difference of the matrices obtained by two different methods
print(np.linalg.norm(ptm_from_choi - ptm_from_kraus))

Matching the outputs of dnorm in chflow and qutip.

In [1]:
import numpy as np
import qutip as qt
import cvxopt as cvx
import picos as pic

In [2]:
def DiamondNorm(choi):
    # computes the diamond norm of the difference between an input Channel and another reference channel, which is by default, the identity channel
    # The semidefinite program outlined in Sec. 4 of DOI: 10.4086/toc.2009.v005a011 is used here.
    # See also: https://github.com/BBN-Q/matlab-diamond-norm/blob/master/src/dnorm.m
    # For some known types of channels, the Diamond norm can be computed efficiently
    # print("Function: dnorm")
    bell = np.zeros((4,4),dtype=np.complex128)
    bell[0,0]=0.5
    bell[0,3]=0.5
    bell[3,0]=0.5
    bell[3,3]=0.5
    diff = np.real((choi - bell + choi.conj().T - bell.conj().T))/2
    #### picos optimization problem
    prob = pic.Problem()
    # variables and parameters in the problem
    J = pic.Constant("J", cvx.matrix(diff))
    Z = pic.Constant("Z", value=0, shape=(2,2))
    rho = pic.HermitianVariable("rho", (2, 2))
    W = pic.HermitianVariable("W", (4, 4))
    # objective function (maximize the hilbert schmidt inner product -- denoted by '|'. Here A|B means trace(A^\dagger * B))
    prob.set_objective("max", J | W)
    # adding the constraints
    prob.add_constraint(W >> 0)
    prob.add_constraint(rho >> 0)
    prob.add_constraint(("I" | rho) == 1)
    prob.add_constraint((W - ((rho & Z) // (Z & rho))) << 0)
    # solving the problem
    prob.solve(verbosity=0, max_iterations=500)
    dnorm = np.double(prob) * 2
    # print("SDP dnorm = %.4e" % (dnorm))
    return dnorm

In [3]:
channel = qt.rand_super(2)
choi_obj = qt.to_choi(channel)
choi_mat = choi_obj.full()/2
print("Random channel:\n{}".format(np.round(choi_mat,2)))

Random channel:
[[ 0.28+0.j    0.02-0.08j -0.06-0.06j  0.14-0.07j]
 [ 0.02+0.08j  0.22+0.j    0.03-0.05j  0.06+0.06j]
 [-0.06+0.06j  0.03+0.05j  0.22+0.j   -0.02+0.08j]
 [ 0.14+0.07j  0.06-0.06j -0.02-0.08j  0.28+0.j  ]]


In [4]:
isherm = np.round(choi_mat - choi_mat.T.conj(),2)
isuntr = np.round(np.trace(choi_mat),2)
eigvals = np.round(np.linalg.eigvals(choi_mat),2)
print("R - R^\dag = {}\nTrace(R) = {}\nEigenvalues = {}".format(isherm, isuntr, eigvals))

R - R^\dag = [[ 0.+0.j  0.+0.j -0.-0.j  0.-0.j]
 [ 0.+0.j  0.+0.j -0.+0.j  0.+0.j]
 [ 0.-0.j  0.+0.j  0.+0.j  0.+0.j]
 [-0.-0.j -0.+0.j  0.+0.j  0.+0.j]]
Trace(R) = (1+0j)
Eigenvalues = [0.55+0.j 0.12+0.j 0.17-0.j 0.16+0.j]


In [5]:
qutip_dnorm = qt.dnorm(choi_obj)

In [ ]:
chflow_dnorm = DiamondNorm(choi_mat)

In [ ]:
print("qutip: {}, chflow = {}. Difference = {}.".format(qutip_dnorm, chflow_dnorm, abs(qutip_dnorm - chflow_dnorm)))